In [ ]:
import os
import random

random.seed(43)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger().setLevel(logging.ERROR)

import torch
import torch.nn as nn
from torchsummary import summary

In [ ]:
# find device
if torch.cuda.is_available(): # NVIDIA
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # apple silicon
    device = torch.device('mps') 
else:
    device = torch.device('cpu') # fallback
device

In [ ]:
num_cpus = os.cpu_count()
print(num_cpus, 'CPUs available')
num_cpus = 1

# Preparing the dataset

In [ ]:
#data directories
dataai="../data/WISDM_ar_v1.1/"

datapath = dataai
col_names = ['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel']

df = pd.read_csv(datapath+"WISDM_ar_v1.1_raw.txt",
                  header=None, names=col_names, delimiter=',', comment=';',
                    on_bad_lines='skip') #skip/warn bad lines
print(df.shape)
df.head()

In [ ]:
num_usrs = df['user'].unique()
print(num_usrs.shape)

In [ ]:
class_labels = df.activity.unique()
num_channels =3 # x-accel, y-accel, z-accel
num_classes = len(class_labels)
print(class_labels)

In [ ]:
activity_to_label = {activity: i for i, activity in enumerate(df['activity'].unique())}
print(activity_to_label)

In [ ]:
# Group by users and activity
grouped = df.groupby(['user', 'activity'])

In [ ]:
# Combine X, Y, Z into a single time-series for each group
time_series = []
labels = []
activity_to_label = {activity: i for i, activity in enumerate(df['activity'].unique())}

for (user, activity), group in grouped:
    # Stack X, Y, Z into a single array of shape (timesteps, 3)
    series = np.column_stack((group['x-accel'], group['y-accel'], group['z-accel']))
    time_series.append(series)
    labels.append(activity_to_label[activity])

print(time_series[0].shape)
print(len(labels), np.unique(labels, return_counts=True))

In [ ]:
ynumpy = np.array(labels)
class_weights=class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(ynumpy), y=ynumpy) 
class_weights=torch.tensor(class_weights,dtype=torch.float)

print(np.unique(ynumpy),class_weights)
print(class_weights.sum(axis=0))

In [ ]:
# Pad/truncate to a fixed length
max_length = 4*1000  # Choose a fixed length
padded_series = nn.utils.rnn.pad_sequence([torch.tensor(series, dtype=torch.float32) for series in time_series],
                             batch_first=True, padding_value=0)
print(padded_series.shape)
padded_series = padded_series[:, :max_length, :]  # Truncate to max_length if necessary


# Normalize (not needed here, but keep it for plotting)
# padded_series = nn.functional.normalize(padded_series, dim=2, p=2)
print(padded_series.shape)


In [ ]:
# Normalized series
num= 178
fig, ax = plt.subplots(5, 1, figsize=(6, 8))
i=0
for num in [0, 50, 100, 150, 170]:
    print(padded_series[num][99])
    ax[i].plot(padded_series[num][:, 0],
            label=f'X:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].plot(padded_series[num][:, 1],
            label=f'Y:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].plot(padded_series[num][:, 2],
            label=f'Z:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].legend()

    i+=1

In [ ]:
# from CNN_LSTM import run_training_testing_cnnlstm

# print(padded_series.shape)
# run_training_testing_cnnlstm.run_cnnlstm_training(padded_series, labels, class_labels, 
#                                                   device, num_channels, num_classes, 
#                                                   test_size=0.45,val_size=0.45, batch_size=4, 
#                                                   num_cpus=1,lr=0.001, num_epochs=10, patience=5, 
#                                                   modeltype = "cnnlstmskip", max_length_series=max_length, num_conv_layers=2, 
#                                                   size_linear_lyr=10, num_blocks_per_layer=2, 
#                                                   initial_channels=8,lstm_hidden_size=16, 
#                                                   lstm_layers=1,opt="adamW", verbose=False, 
#                                                   pathsave="./Figs", weights=None, norm_type="per-channel")

In [ ]:
# from CNN_LSTM.nn_CNN_model import CNNI

# model = CNNI(3, num_classes, max_length, num_conv_layers=3, size_linear_lyr=16)
# # Define the input shape (input_channels, timesteps)
# input_shape = (max_length, 3)  # max_lenth, 3 channels (X, Y, Z)
# print(summary(model, input_size=input_shape))

In [ ]:
# from CNN_LSTM.nn_CNN_model import CNNSkipConnections

# model = CNNSkipConnections(3, num_classes=num_classes, num_layers=2, num_blocks_per_layer=1, initial_channels=8)
# # Define the input shape (input_channels, timesteps)
# input_shape = (max_length, 3)  # max_lenth, 3 channels (X, Y, Z)
# print(summary(model, input_size=input_shape))

In [16]:
from CNN_LSTM import run_training_testing_cnnlstm

# print(padded_series.shape)
run_training_testing_cnnlstm.run_cnnlstm_training(padded_series, labels, class_labels, 
                                                  device, num_channels, num_classes, 
                                                  test_size=0.2,val_size=0.2, batch_size=4, 
                                                  num_cpus=1,lr=0.00001, num_epochs=10, patience=5, 
                                                  modeltype = "cnnlstmskip", max_length_series=max_length, 
                                                  num_conv_layers=4, size_linear_lyr=10, num_blocks_per_layer=2, 
                                                  initial_channels=10,lstm_hidden_size=4, 
                                                  num_lstm_layers=3,opt="adamW", verbose=False, 
                                                  pathsave="./Figs/", weights=None, norm_type="per-channel")